# nlp-pandas
Below is a rough, under construction proposal on a package that would take natural language and query a pandas DataFrame after structuring the command.

Thoughts on a data analytics add on that can answer questions on custom datasets using NLP (or even just structuring without intelligence at first.

The idea is to complement / replace data viz platforms like Tableau -- instead of users having to generate the tables they want, they should be able to ask free form questions.

The original idea was nlp-sql, but I think nlp-pandas may work better to start out with. pandas has a lot of built in functionality that may come in handy.

## Sample of how it could work

 - users_table = ['id', 'f_name', 'l_name', 'city', 'state']
 - orders_table = ['id', 'user_id', 'order_date', 'payment_method']
 - items_table = ['id', 'product_id', 'order_id', 'count']
 - products_table = ['id', 'name', 'price']

### Questions:

 - Number of orders placed in the last six months
 - Bar chart of orders placed in the last six months by month
 - Number of orders placed in the last six months by state
 - Top 10 orders by value
 - Top 10 states by revenue
 - Top 10 products by revenue this month
 - Top 10 products by revenue by month for the last six months
 
We would have to understand how we can translate this to SQL. Perhaps we can use pandas first for now. We'd have to calculate intermediate fields. We'd also have to understand table linkages. Perhaps to start, we can denormalize tables and just run the queries on single tables. Later, we can start to break these out.

Changing name to `nlp-pandas`.



### Proposal

```python
import nlp-pandas as nd
import pandas as pd

df_users, df_orders, df_items, df_products

ds = nd.Dataset(df_users, df_orders, df_items, df_products) # set the dataset
df_answer = ds.ask('Top 10 products by revenue by month for the last six months')
```

df_answer is the result of the last line. 

### How do we get it working?

The first way is to hard code stuff in. This is how Alexa works. For example, if we see 'top' or 'largest' or 'biggest' followed by a number, we know to sort desc and limit the number. If we see 'by' followed by this, we know what column to sort using.

It is easiest to first work on a single table. Later, it might be possible to join tables intelligently.

## v1

In [4]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

In [24]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [20]:
def ask(df, question):
    words = question.split()
    q ={}
    for w in words:
        if w in ['top', 'largest', 'first']: q['ascending'] = False
        if w in ['bottom', 'smallest', 'last']: q['direction'] = True
        if (w.isdigit()): q['limit'] = int(w)
        if w in df.columns: q['column'] = w
    return df.sort_values(q['column'], ascending=q['ascending'])[:limit]

In [21]:
question = 'top 10 sepal_length'

In [22]:
ask(df, 'top 10 sepal_length')

,sepal_length,sepal_width,petal_length,petal_width,species
131,7.9,3.8,6.4,2.0,virginica
135,7.7,3.0,6.1,2.3,virginica
122,7.7,2.8,6.7,2.0,virginica
117,7.7,3.8,6.7,2.2,virginica
118,7.7,2.6,6.9,2.3,virginica
105,7.6,3.0,6.6,2.1,virginica
130,7.4,2.8,6.1,1.9,virginica
107,7.3,2.9,6.3,1.8,virginica
125,7.2,3.2,6.0,1.8,virginica
109,7.2,3.6,6.1,2.5,virginica
